In [39]:
import os
import numpy as np 
import pandas as pd 
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
"import plotly.express as px"
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows",None)
from sklearn import preprocessing
import matplotlib 
matplotlib.style.use('ggplot')
from sklearn.preprocessing import LabelEncoder

df=pd.read_csv("C:/Users/mathi/Documents/SMART/SmartFinal/heart_10000.csv")
df.head()
string_col = df.select_dtypes(include="object").columns
df[string_col]=df[string_col].astype("string")
df.dtypes
# df_tree=pd.get_dummies(df,columns=string_col,drop_first=False)
# df_tree.head()

df_tree = df.apply(LabelEncoder().fit_transform)
df_tree.head()

,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
0,9,1,1,31,141,0,2,25,0,10,2,0
1,20,0,0,39,72,0,1,34,1,25,1,1
2,26,1,2,49,53,0,1,48,0,10,2,0
3,11,1,2,22,189,0,1,96,0,10,2,0
4,17,0,1,31,95,0,1,96,0,10,2,0


Le but de cette partie est de repérer les valeurs incohérentes dans le jeu de donnée (les strings). Pour ça, on encode les valeurs strings, ensuite, on cherche à parcourir les facteurs, et à compter le nombre de certaines valeurs. Ensuite, on fait un rapport entre chaque nombre de fois qu'on a trouvé une valeur par rapport aux autres. Un rapport très bas pourra indiquer deux choses différentes : soit cette valeur est une erreur et est incohérente par rapport au reste, soit le peu de valeurs de ce type baisse la qualité du jeu de donnée car rend plus compliqué son interprétation correcte et la mesure de sa corrélation avec les autres valeurs. Dans tout les cas, cela baisse la qualité de notre jeu de donnée.

In [ ]:
def calcul_prop (df, col) :
    prop = df[col].value_counts(normalize=True)
    prop = prop.reset_index()
    prop.columns = [col, "proportion"]
    prop["rapport"] = 0.0
    return prop

print(f"nombre de lignes: {len(df)}")

cpt = 0

for col in df_tree.columns:
    proportion = calcul_prop(df_tree, col)
    for i in range(len(proportion)):
        autres_proportions = proportion["proportion"].drop(i)
        if len(autres_proportions) > 0:
            moyenne = autres_proportions.mean()
            ecart_type_corrige = autres_proportions.std(ddof=1)
            z = (proportion.at[i, "proportion"] - moyenne) / ecart_type_corrige
            proportion.at[i, "rapport"] = z
            if z < -2:
                cpt += 1
    
    print(f"\nAnalyse de la colonne: {col}")
    print(proportion.sort_values("rapport"))

print(f"Nombre de valeurs aberrantes: {cpt}")
print(f"Proportion de valeurs aberrantes: {cpt / len(df_tree) * 100:.2f}%")
    





nombre de lignes: 9998

Analyse de la colonne: 49
    49  proportion   rapport
48   0    0.001100 -1.323522
49  45    0.001100 -1.323522
47   2    0.001100 -1.323522
45  49    0.002200 -1.243963
44   5    0.002200 -1.243963
43   3    0.002200 -1.243963
46  48    0.002200 -1.243963
42   1    0.003201 -1.172042
41  47    0.003301 -1.164870
40  44    0.004401 -1.086208
39  43    0.005401 -1.015045
38   4    0.005501 -1.007946
37   8    0.006601 -0.930052
35  42    0.007702 -0.852497
34  46    0.007702 -0.852497
36   6    0.007702 -0.852497
33  40    0.010802 -0.635496
32   7    0.011902 -0.558962
31   9    0.012002 -0.552014
29  12    0.014203 -0.399545
30  38    0.014203 -0.399545
28  41    0.014303 -0.392630
27  11    0.016203 -0.261433
26  39    0.016303 -0.254536
25  10    0.017504 -0.171833
24  17    0.019604 -0.027254
23  14    0.019804 -0.013489
22  16    0.020404  0.027805
21  19    0.020704  0.048452
20  37    0.022805  0.193052
19  21    0.022905  0.199942
18  36    0.023905  0.